In [1]:
!pip install pytorch_lightning
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import pandas_gbq as gbq

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, PMM, GMM, NBMM
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic

In [ ]:
train  = pd.read_csv('/content/drive/MyDrive/Vertex AI/NHints/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Vertex AI/NHints/Test.csv')

# train_query="""SELECT * EXCEPT(row_num, splits,is_test)
# FROM `ai-ps6060.Vertex_AI_Experiment.tapestry_cnbo_l4_nhits_train`"""
# train = gbq.read_gbq(train_query, project_id='ai-ps6060')

# test_query="""SELECT * EXCEPT(row_num, splits,is_test)
# FROM `ai-ps6060.Vertex_AI_Experiment.tapestry_cnbo_l4_nhits_test`"""
# test = gbq.read_gbq(test_query, project_id='ai-ps6060')

In [ ]:
Y_train_df = train
Y_test_df = test

model = NHITS(h=52,
              input_size=105,
              loss=DistributionLoss(distribution='StudentT', level=[80, 90], return_params=True),
              #loss=DistributionLoss(distribution='Normal', level=[80, 90], return_params=True),
              #loss=DistributionLoss(distribution='Poisson', level=[80, 90], return_params=True),
              #loss=DistributionLoss(distribution='Tweedie', level=[80, 90], rho=1.5),
              #loss=DistributionLoss(distribution='NegativeBinomial', level=[80, 90], return_params=True),
              #loss=NBMM(n_components=2, level=[80,90]),
              #loss=GMM(n_components=2, level=[80,90]),
              #loss=PMM(n_components=1, level=[80,90]),
              #stat_exog_list=['D13-CHM-CHM-MTL'],
              #futr_exog_list=['trend'],
              n_freq_downsample=[2, 1, 1],
              scaler_type='robust',
              max_steps=200,
              early_stop_patience_steps=2,
              inference_windows_batch_size=1,
              val_check_steps=10,
              learning_rate=1e-3)



INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
#Column Rename
Y_train_df = Y_train_df.rename(columns={'l4_name': 'unique_id', 'date': 'ds', 'qty':'y'})
Y_test_df = Y_test_df.rename(columns={'l4_name': 'unique_id', 'date': 'ds', 'qty':'y'})

In [ ]:
#Change datatype
Y_train_df['ds'] = pd.to_datetime(Y_train_df['ds'])
Y_test_df['ds'] = pd.to_datetime(Y_test_df['ds'])

In [ ]:
#Column Drop
Col_drop = ['l3_name','l2_name','rev_flag']
Y_train_df = Y_train_df.drop(columns = Col_drop)
Y_test_df = Y_test_df.drop(columns = Col_drop)

In [ ]:
Y_train_df = pd.get_dummies(Y_train_df, columns=['events'], prefix='',)
Y_test_df = pd.get_dummies(Y_test_df, columns=['events'], prefix='')

In [ ]:
Y_train_df.info()
Y_train_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42038 entries, 0 to 42037
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   unique_id              42038 non-null  object        
 1   fiscal_year_week       42038 non-null  int64         
 2   ds                     42038 non-null  datetime64[ns]
 3   y                      42038 non-null  float64       
 4   msrp                   42038 non-null  float64       
 5   percent_off            42038 non-null  float64       
 6   fiscal_bi_week         42038 non-null  float64       
 7   fiscal_quarter         42038 non-null  int64         
 8   _-                     42038 non-null  uint8         
 9   _ada_event_1           42038 non-null  uint8         
 10  _christmas             42038 non-null  uint8         
 11  _columbus_day          42038 non-null  uint8         
 12  _cyber_monday          42038 non-null  uint8         
 13  _

(42038, 36)

In [ ]:
Y_train_df

,unique_id,fiscal_year_week,ds,y,msrp,percent_off,fiscal_bi_week,fiscal_quarter,_-,_ada_event_1,...,_pre_fathers_day,_pre_labor_day,_pre_memorial_day,_pre_mothers_day,_pre_thanks_giving,_presidents_day,_st_patricks_day,_thanks_giving,_valentines_day,_womens_day
0,D13-CHM-CHM-MTL,202240,2022-04-03,29.000000,45.10,0.74,20.0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
1,D22-GLO-GLO-GLO,202240,2022-04-03,21.000000,150.41,0.71,20.0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
2,D08-OTH-OTH-OTH,202240,2022-04-03,0.000000,29.94,0.63,20.0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
3,D16-GLO-GLO-LTH,202240,2022-04-03,0.000000,182.62,0.52,20.0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
4,D16-SCA-SCA-WOV,202240,2022-04-03,2.000000,180.79,0.71,20.0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42033,D23-TEC-TEC-SPP,202237,2022-03-13,2.000000,75.50,0.82,19.0,3,0,0,...,0,0,0,0,0,0,1,0,0,0
42034,D23-TEC-TEC-SPP,202238,2022-03-20,2.000000,73.30,0.69,19.0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
42035,D23-TEC-TEC-PHC,202238,2022-03-20,483.357692,59.34,0.34,19.0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
42036,D23-TEC-TEC-SPP,202239,2022-03-27,2.000000,72.66,0.73,20.0,3,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

Y_test_df

,unique_id,fiscal_year_week,ds,y,msrp,percent_off,fiscal_bi_week,fiscal_quarter,_-,_ada_event_1,...,_pre_fathers_day,_pre_labor_day,_pre_memorial_day,_pre_mothers_day,_pre_thanks_giving,_presidents_day,_st_patricks_day,_thanks_giving,_valentines_day,_womens_day
0,D22-BEL-BEL-SZB,202240,2022-04-03,0.000000,150.00,0.52,20.0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
1,D22-BEL-BEL-SZB,202241,2022-04-10,0.000000,150.00,0.55,21.0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
2,D22-BEL-BEL-SZB,202242,2022-04-17,1.000000,150.00,0.71,21.0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
3,D22-BEL-BEL-SZB,202243,2022-04-24,1.000000,150.00,0.75,22.0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
4,D22-BEL-BEL-SZB,202244,2022-05-01,0.000000,150.00,0.75,22.0,4,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12007,D23-TEC-TEC-PHC,202351,2023-06-18,786.808729,61.82,0.53,26.0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
12008,D23-TEC-TEC-SPP,202352,2023-06-25,0.000000,74.38,0.57,26.0,4,0,0,...,1,0,0,0,0,0,0,0,0,0
12009,D23-TEC-TEC-PHC,202352,2023-06-25,881.557585,62.33,0.58,26.0,4,0,0,...,1,0,0,0,0,0,0,0,0,0
12010,D23-TEC-TEC-PHC,202401,2023-07-02,946.208511,61.58,0.55,1.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Y_test_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12012 entries, 0 to 12011
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   unique_id              12012 non-null  object        
 1   fiscal_year_week       12012 non-null  int64         
 2   ds                     12012 non-null  datetime64[ns]
 3   y                      12012 non-null  float64       
 4   msrp                   12012 non-null  float64       
 5   percent_off            12012 non-null  float64       
 6   fiscal_bi_week         12012 non-null  float64       
 7   fiscal_quarter         12012 non-null  int64         
 8   _-                     12012 non-null  uint8         
 9   _ada_event_1           12012 non-null  uint8         
 10  _christmas             12012 non-null  uint8         
 11  _columbus_day          12012 non-null  uint8         
 12  _cyber_monday          12012 non-null  uint8         
 13  _

In [ ]:
model = NHITS(h=52,
              input_size=52,
              loss=DistributionLoss(distribution='StudentT', level=[80, 90], return_params=True),
              #loss=DistributionLoss(distribution='Normal', level=[80, 90], return_params=True),
              #loss=DistributionLoss(distribution='Poisson', level=[80, 90], return_params=True),
              #loss=DistributionLoss(distribution='Tweedie', level=[80, 90], rho=1.5),
              #loss=DistributionLoss(distribution='NegativeBinomial', level=[80, 90], return_params=True),
              #loss=NBMM(n_components=2, level=[80,90]),
              #loss=GMM(n_components=2, level=[80,90]),
              #loss=PMM(n_components=1, level=[80,90]),
              #stat_exog_list=['D13-CHM-CHM-MTL'],
              #futr_exog_list=['trend'],
              #n_freq_downsample=[2, 1, 1],
              scaler_type='robust',
              max_steps=200,
            #   early_stop_patience_steps=2,
            #   inference_windows_batch_size=1,
            #   val_check_steps=10,
            #   learning_rate=1e-3
              )


INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
fcst = NeuralForecast(models=[model], freq='W')
fcst.fit(df=Y_train_df,  val_size=52)
forecasts = fcst.predict(futr_df=Y_test_df)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# forecasts.info()
df_reset = forecasts.reset_index()
df_reset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12012 entries, 0 to 12011
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   unique_id     12012 non-null  object        
 1   ds            12012 non-null  datetime64[ns]
 2   NHITS         12012 non-null  float64       
 3   NHITS-median  12012 non-null  float64       
 4   NHITS-lo-90   12012 non-null  float64       
 5   NHITS-lo-80   12012 non-null  float64       
 6   NHITS-hi-80   12012 non-null  float64       
 7   NHITS-hi-90   12012 non-null  float64       
 8   NHITS-df      12012 non-null  float64       
 9   NHITS-loc     12012 non-null  float64       
 10  NHITS-scale   12012 non-null  float64       
dtypes: datetime64[ns](1), float64(9), object(1)
memory usage: 1.0+ MB


In [ ]:
df_reset.to_gbq("Vertex_AI_Experiment.tapestry_cnbo_l4_nhits_results_trail_1", project_id="ai-ps6060", if_exists='replace')



100%|██████████| 1/1 [00:00<00:00, 6043.67it/s]


In [ ]:
# Result -> val_size = 12
# Result_v1 -> val_size = 52
# Result_v2 -> val_size = 12, one_hot encoded column
# Result_v3 -> val_size = 52, one_hot encoded column
